In [1]:
import qiniu
import time
import requests

初始化，获取签名授权

In [2]:
access_key = "x34GCNDuYpCFsNtqkrri3Eagr6OANYyboWRZtWQv"
secret_key = "f_lfzdkBoEfOa_7a_0yrMOcIdfr-cHv9H463B5Xh"
# access_key = "MUpLpPtmVeiP7UWWteuSRBm5cHNeqSMndp4zh339"
# secret_key = "0ix4DVQZvPdlQNDIn0qTudaBj8h27FK9qWThMd5L"

# 构建鉴权对象
q = qiniu.Auth(access_key, secret_key)

数据或文件上传

In [7]:
# data upload
# key = 'pupy dog here.mp4'
# data = 'hello qiniu!'
# token = q.upload_token("yy-short-video")
# ret, info = qiniu.put_data(token, key, data)

# file upload
token = q.upload_token("yy-short-video")
key = 'user2/pupy dog here.mp4'
file_path = './data/upload/1185308075.mp4'
ret, info = qiniu.put_file(token, key, file_path, metadata=dict(name="tiantian"))
if ret is not None:
    print('All is OK', ret)
else:
    print(info)

All is OK {'hash': 'FkwkxLPsVhf1C3FK7EsSVprQLktO', 'key': 'user2/pupy dog here.mp4'}


资源管理

In [33]:
# 初始化 BucketManager 空间管理类
bucket = qiniu.BucketManager(q)

# 指定测试空间与文件名
bucket_name = "yy-short-video"
key = "user@1/video/5847849856.mp4"

ret, info = bucket.stat(bucket_name, key)
# ret, info = bucket.delete(bucket_name, key)
info

_ResponseInfo__response:<Response [200]>, exception:None, status_code:200, text_body:, req_id:A5UAAADdLNV9OJMX, x_log:-

In [90]:
prefix = None
# 列举条目
limit = 1
# 列举出除'/'的所有文件以及以'/'为分隔的所有前缀
delimiter = None
# 标记
marker = None
ret, eof, info = bucket.list(bucket_name, prefix, marker, limit, delimiter)
print(ret.get('items'))

[{'key': 'pdg.mp4', 'hash': 'FkwkxLPsVhf1C3FK7EsSVprQLktO', 'fsize': 1929256, 'mimeType': 'video/mp4', 'putTime': 16985474640459057, 'type': 0, 'status': 0, 'md5': '1954158dfcc00fa37bca49bbda9cfc2a', 'x-qn-meta': {'hhh': '123', 'nihao': 'aaa'}}]


In [106]:
bucket_domains = bucket.list_domains(bucket_name)
print(q.private_download_url(f"http://{bucket_domains[0][0]}/pdg.mp4"))
unix_time = ret.get('items')[0].get('putTime') // (10**7)
time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(unix_time))

'2023/10/29 10:44:24'

资源处理

In [12]:
# 是使用的队列名称,不设置代表不使用私有队列，使用公有队列。
pipeline = "small_eye"

# 要进行转码的转码操作。
fops = "vframe/jpg/offset/7/w/480/h/360"

saveas_key = qiniu.urlsafe_base64_encode(f"{bucket_name}:{key.split('.')[0]}_frame7.jpg")
fops = fops+"|saveas/"+saveas_key

pfop = qiniu.PersistentFop(q, bucket_name, pipeline)
ops = [fops]
ret, info = pfop.execute(key, ops, 1)

In [13]:
ret

{'persistentId': 'cn-east-2.01c201cwi9wdvuim3c00mvdajq000akz'}

In [4]:
url = "http://s31euirdk.bkt.clouddn.com/user%401/manifest/video/5847849856.mp4.json?e=1698758918&token=x34GCNDuYpCFsNtqkrri3Eagr6OANYyboWRZtWQv:_qNIXyubHGVZO_CHyqwkDzf43Yg="
data = requests.get(url).json()
data

{'username': 'y',
 'user_id': '1',
 'title': '狗狗真可爱',
 'submit_time': '12:11',
 'midea_type': 'video'}